In [ ]:
import os
import shutil
from ultralytics import YOLO
import torch
from statistics import mean

In [ ]:
folders = ["datasets_b", "datasets_m", "datasets_c", "datasets_o", "datasets_p", "datasets_s"]
base_directory = os.getcwd()

def rotate_working_folder(folders_array, base_path):
    missing_name = None
    missing_index = -1
    
    for i, folder_name in enumerate(folders_array):
        folder_path = os.path.join(base_path, folder_name)
        if not os.path.exists(folder_path):
            missing_name = folder_name
            missing_index = i
            break
    
    if missing_name is None:
        print("All folders exist - no changes needed")
        return
    
    print(f"Found missing folder: {missing_name} at position {missing_index}")
    x_folder = "datasets" 
    next_index = (missing_index + 1) % len(folders_array)
    next_name = folders_array[next_index]
    next_path = os.path.join(base_path, next_name)
    
    if not os.path.exists(next_path):
        print(f"Next folder {next_name} also missing - cannot perform rotation")
        return

    temp_name = f"{x_folder}_temp"
    temp_path = os.path.join(base_path, temp_name)
    
    try:
        shutil.move(next_path, temp_path)
        print(f"Renamed {next_name} to temporary {temp_name}")
        
        x_path = os.path.join(base_path, x_folder)
        if os.path.exists(x_path):
            shutil.move(x_path, os.path.join(base_path, missing_name))
            print(f"Renamed {x_folder} to {missing_name}")
        
        shutil.move(temp_path, os.path.join(base_path, x_folder))
        print(f"Renamed {temp_name} to {x_folder}")
        
        print("Rotation completed successfully")
    except Exception as e:
        print(f"Error during renaming: {e}")
        if os.path.exists(temp_path):
            shutil.move(temp_path, next_path)
    return missing_name

In [ ]:
model = YOLO("yolo11l.pt")

In [ ]:
torch.cuda.empty_cache()
print(torch.cuda.is_available())
torch.cuda.get_device_name(0)

In [ ]:
metrics_dict = dict()
arr_name = []
arr_v = []
for i in range(6):
    model = YOLO("yolo11l.pt")
    model.train(data="datasets/data.yaml", epochs=300, imgsz=640, batch = 4, device = 0, patience = 5 )
    vals = model.val()
    hc = rotate_working_folder(folders, base_directory)
    metrics_dict[hc] = vals
    arr_name.append(hc)
    arr_v.append(vals)

In [ ]:
names = {"datasets_b": "oreo", "datasets_m": "milk", "datasets_c": "coffee", "datasets_o":"oil", "datasets_p":"pocky", "datasets_s":"candy"}
for e in folders:
    print(names[e])
    print(metrics_dict[e].box.map50)
    print(metrics_dict[e].box.map)
    print(mean(metrics_dict[e].box.p))
    print(mean(metrics_dict[e].box.r))
    print("-------------------")